In [9]:
import rootutils
root_path = rootutils.setup_root(".", indicator=".project-root", pythonpath=True)

In [10]:
import torch
from core.GPTModel import GPTModel

In [11]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [12]:
GPT_CONFIG_124M = {
 "vocab_size": 50257, # Vocabulary size
 "context_length": 256, # Context length
 "emb_dim": 768, # Embedding dimension
 "n_heads": 12, # Number of attention heads
 "n_layers": 12, # Number of layers
 "dropout": 0.1, # Dropout rate
 "qvk_bias": False
}

In [13]:
file_path = 'C:/Users/WalterBuenodeBritoNe/Documents/GitHub/LLMPlayground/Data/the-verdict.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

print(text[:100])

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [14]:
total_characters = len(text)
total_tokens = len(tokenizer.encode(text))
print("Characters:", total_characters)
print("Tokens:", total_tokens)

Characters: 20479
Tokens: 5145


In [15]:
train_ratio = 0.9
train_size = int(train_ratio * len(text))
train_text = text[:train_size]
validation_text = text[train_size:]

print(f"Train size: {len(train_text)}")
print(f"Validation size: {len(validation_text)}")

train_tokens = tokenizer.encode(train_text)
validation_tokens = tokenizer.encode(validation_text)

print(f"Train tokens: {len(train_tokens)}")
print(f"Validation tokens: {len(validation_tokens)}")

Train size: 18431
Validation size: 2048
Train tokens: 4612
Validation tokens: 534


In [16]:
from core.DataUtils import gpt_dataloader

In [17]:
from core.DataUtils.gpt_dataloader import create_dataloader_v1

train_loader = create_dataloader_v1(
 train_text,
 batch_size=2,
 max_length=GPT_CONFIG_124M["context_length"],
 stride=GPT_CONFIG_124M["context_length"],
 drop_last=True,
 shuffle=True,
 num_workers=0
)

validation_loader = create_dataloader_v1(
 validation_text,
 batch_size=2,
 max_length=GPT_CONFIG_124M["context_length"],
 stride=GPT_CONFIG_124M["context_length"],
 drop_last=True,
 shuffle=True,
 num_workers=0
)

In [18]:
print("Train loader:")
for x, y in train_loader:
 print(x.shape, y.shape)
print("\nValidation loader:")
for x, y in validation_loader:
 print(x.shape, y.shape)

Train loader:
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])

Validation loader:
torch.Size([2, 256]) torch.Size([2, 256])


In [20]:
from core.Train import GPTTrainer

In [23]:
torch.manual_seed(123)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(
 model.parameters(),
 lr=0.0004, weight_decay=0.1
)
num_epochs = 10

In [ ]:
trainer = GPTTrainer(model, epochs=num_epochs, eval_freq=5, eval_iter=5, train_loader=train_loader, validation_loader=validation_loader, optimizer=optimizer, device=device)
trainer.train()